In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import math

def import_facebook_data(path):
    df = pd.read_csv(path, sep=' ', header=None)
    return df.values

In [31]:
def deltaQ(node1, cluster_value_from_node1, y, adj, cluster, m):
    nodes_x = set()
    nodes_x.update(cluster[cluster_value_from_node1])
    nodes_y = set()
    nodes_y.update(cluster[y])
    
    sigma_x = 0
    for v in cluster[x]:
        sigma_x += len(adj[v])
    
    sigma_y = 0
    for v in cluster[y]:
        sigma_y += len(adj[v])
    
    deg_u = len(adj[u])
    # edges of u in x
    e_ux = 0
    # edges of u in y
    e_uy = 0
    for v in adj[node1]:
        if v in nodes_x:
            e_ux += 1
        elif v in nodes_y:
            e_uy += 1
   
    # dQx is change in modularity of x after u is removed
    # dQy is change in modularity of y after u is added

    dQx = -2 * e_ux / (2 * m)
    dQy = 2 * e_uy / (2 * m)
    
    m4m = 4 * m * m
    dQx = dQx + (sigma_x ** 2 - (sigma_x - deg_u) ** 2) / m4m
    dQy = dQy + (sigma_y ** 2 - (sigma_y + deg_u) ** 2) / m4m
    
    # returning total modularity change
    return dQx + dQy

In [30]:
def louvain_one_iter(nodes_connectivity_list):
    number_of_edges = len(nodes_connectivity_list)/2
    # adjacency list
    adjacency_list = {}

    for edge in nodes_connectivity_list:
        if edge[0] not in adjacency_list:
            adjacency_list[edge[0]] = []
        adjacency_list[edge[0]].append(edge[1])
    
    #Initial cluster value of each node to itself
    cluster_value_from_node = {}
    for node in adjacency_list:
        cluster_value_from_node[node] = node
    
    # initial modularity
    modularity = 0
    for node in adjacency_list:
        modularity += len(adjacency_list[node]) ** 2
        
    modularity = -modularity /( 4 *number_of_edges *number_of_edges)
    
    print("Initial modularity = ", modularity)
    
    nodes = list(adjacency_list.keys())
    # Iterating until modularity is saturated
    for x in range(10):
        count = 0

        for node1 in nodes:
        
            #######
            # cluster to node
            node_value_from_cluster = {}
            for node in cluster_value_from_node:
                if node_value_from_cluster.get(cluster_value_from_node[node]) == None:
                    node_value_from_cluster[cluster_value_from_node[node]] = []
                node_value_from_cluster[cluster_value_from_node[node]].append(node)

            # track the cluster _c where node _u belongs ((u, _u) element of E)
            # such that delta is maximum so far if node u 
            # is moved to cluster _c
            node2 = -1
            new_cluster = -1
            delta = -math.inf
            ######
            
            for node in adjacency_list[node1]:
                c = cluster_value_from_node[node]
                # if in same cluster or no edge of u with target cluster c
                if cluster_value_from_node[node1] == c:
                    continue
                    
                delQ = deltaQ(node1, cluster_value_from_node[node1], c, adjacency_list, node_value_from_cluster, number_of_edges)
                if delQ > delta:
                    delta = delQ
                    node2 = node1
                    new_cluster = c
            

            if modularity + delta <= modularity:
                count += 1
                continue
            
            # updates Q when new modularity is more than previous
            modularity = modularity + delta
            cluster_value_from_node[node2] = new_cluster
        print(f"Iteration = {x}, Modularity = {modularity}")
        if count == len(nodes):
            break
    
    print(modularity)
    
    # for each cluster, the smallest node it mapped to
    node_value_from_cluster = {}
    for key in cluster_value_from_node:
        if node_value_from_cluster.get(cluster_value_from_node[key]) == None:
            node_value_from_cluster[cluster_value_from_node[key]] = key
        else:
            node_value_from_cluster[cluster_value_from_node[key]] = min(key, node_value_from_cluster[cluster_value_from_node[key]])
    
    # print("Total Partitions = ", len(c2n.keys()))
    n = len(nodes)
    partition = np.zeros((n, 2))
    for i in range(n):
        partition[i][0] = nodes[i]
        partition[i][1] = node_value_from_cluster[cluster_value_from_node[nodes[i]]]
        
    return partition

In [28]:
def louvain_one_iter(nodes_connectivity_list):
    number_of_edges = len(nodes_connectivity_list)/2
    # adjacency list
    adjacency_list = {}

    for edge in nodes_connectivity_list:
        if edge[0] not in adjacency_list:
            adjacency_list[edge[0]] = []
        adjacency_list[edge[0]].append(edge[1])
    
    #Initial cluster value of each node to itself
    cluster_value_from_node = {}
    for node in adjacency_list:
        cluster_value_from_node[node] = node
    
    # initial modularity
    modularity = 0
    for node in adjacency_list:
        modularity += len(adjacency_list[node]) ** 2
        
    modularity = -modularity /( 4 *number_of_edges *number_of_edges)
    
    print("Initial modularity = ", modularity)
    
    nodes = list(adjacency_list.keys())
    # Iterating until modularity is saturated
    for x in range(10):
        count = 0

        for node1 in nodes:
        
            #######
            # cluster to node
            node_value_from_cluster = {}
            for node in cluster_value_from_node:
                if node_value_from_cluster.get(cluster_value_from_node[node]) == None:
                    node_value_from_cluster[cluster_value_from_node[node]] = []
                node_value_from_cluster[cluster_value_from_node[node]].append(node)

            # track the cluster _c where node _u belongs ((u, _u) element of E)
            # such that delta is maximum so far if node u 
            # is moved to cluster _c
            node2 = -1
            new_cluster = -1
            delta = -math.inf
            ######
            
            for node in adjacency_list[node1]:
                c = cluster_value_from_node[node]
                # if in same cluster or no edge of u with target cluster c
                if cluster_value_from_node[node1] == c:
                    continue
                    
                delQ = deltaQ(node1, cluster_value_from_node[node1], c, adjacency_list, node_value_from_cluster, number_of_edges)
                if delQ > delta:
                    delta = delQ
                    node2 = node1
                    new_cluster = c
            

            if modularity + delta <= modularity:
                count += 1
                continue
            
            # updates Q when new modularity is more than previous
            modularity = modularity + delta
            cluster_value_from_node[node2] = new_cluster
        print(f"Iteration = {x}, Modularity = {modularity}")
        if count == len(nodes):
            break
    
    print(modularity)
    
    # for each cluster, the smallest node it mapped to
    node_value_from_cluster = {}
    for key in cluster_value_from_node:
        if node_value_from_cluster.get(cluster_value_from_node[key]) == None:
            node_value_from_cluster[cluster_value_from_node[key]] = key
        else:
            node_value_from_cluster[cluster_value_from_node[key]] = min(key, node_value_from_cluster[cluster_value_from_node[key]])
    
    # print("Total Partitions = ", len(c2n.keys()))
    n = len(nodes)
    partition = np.zeros((n, 2))
    for i in range(n):
        partition[i][0] = nodes[i]
        partition[i][1] = node_value_from_cluster[cluster_value_from_node[nodes[i]]]
        
    return partition

In [29]:

nodes_connectivity_list_fb = import_facebook_data("../data/facebook_combined.txt")

## add reverse edges to make undirected graph
reverse_edges = np.flip(nodes_connectivity_list_fb, axis=1)
nodes_connectivity_list_fb = np.concatenate((nodes_connectivity_list_fb, reverse_edges), axis=0)

graph_partition_louvain_fb = louvain_one_iter(nodes_connectivity_list_fb)
np.savetxt("graph_partition_louvain_fb.txt", graph_partition_louvain_fb, fmt='%d')

Initial modularity =  -0.0006039046004050387
Iteration = 0, Modularity = 0.6175247933526578
Iteration = 1, Modularity = 0.77366715604682
Iteration = 2, Modularity = 0.7958870434416259
Iteration = 3, Modularity = 0.8065724798305722
Iteration = 4, Modularity = 0.8102778992022992
Iteration = 5, Modularity = 0.8105311539272722
Iteration = 6, Modularity = 0.8105424154958313
Iteration = 7, Modularity = 0.8105424154958313
0.8105424154958313
